In [17]:
import mlflow
import os
import yaml

In [18]:
hyperparams = {
    "resume_checkpoint": "None",  # "restored_model_checkpoint/checkpoint_epoch=06.ckpt",
    "resume_run_id": "None",  # "ef539b4138fa4055bf65c58f30249211",
    "max_epochs": 10,
    "gpus": 0,
    "batch_size": 32,
    "lr": 0.0001,
    "num_samples": -1,
    "val_ratio": 0.2,
    "test_ratio": 0,
    "random_seed": "None",
    # "dataset": "/FileStore/tables/datasets/dummy.json",
    "dataset": "data/processed/dummy.json",
}

In [19]:
# Two options: None to run locally or configure a cluster to run on databricks
# https://docs.databricks.com/dev-tools/api/latest/clusters.html
# Change the configuration accordingly to AWS, Azure, Google Cloud
backend_config = None
"""
backend_config = {
    "num_workers": 0,
    "spark_version": "10.4.x-scala2.12",
    "spark_conf": {
        "spark.databricks.cluster.profile": "singleNode",
        "spark.master": "local[*, 4]",
    },
    "azure_attributes": {
        "first_on_demand": 1,
        "availability": "ON_DEMAND_AZURE",
        "spot_bid_max_price": -1,
    },
    "node_type_id": "Standard_F4",
    "driver_node_type_id": "Standard_F4",
    "ssh_public_keys": [],
    "custom_tags": {"ResourceClass": "SingleNode"},
    "spark_env_vars": {"PYSPARK_PYTHON": "/databricks/python3/bin/python3"},
    "enable_elastic_disk": "true",
    "cluster_source": "UI",
    "init_scripts": [],
    "cluster_id": "0519-085431-rvfc7gfq",
}
"""

'\nbackend_config = {\n    "num_workers": 0,\n    "spark_version": "10.4.x-scala2.12",\n    "spark_conf": {\n        "spark.databricks.cluster.profile": "singleNode",\n        "spark.master": "local[*, 4]",\n    },\n    "azure_attributes": {\n        "first_on_demand": 1,\n        "availability": "ON_DEMAND_AZURE",\n        "spot_bid_max_price": -1,\n    },\n    "node_type_id": "Standard_F4",\n    "driver_node_type_id": "Standard_F4",\n    "ssh_public_keys": [],\n    "custom_tags": {"ResourceClass": "SingleNode"},\n    "spark_env_vars": {"PYSPARK_PYTHON": "/databricks/python3/bin/python3"},\n    "enable_elastic_disk": "true",\n    "cluster_source": "UI",\n    "init_scripts": [],\n    "cluster_id": "0519-085431-rvfc7gfq",\n}\n'

In [20]:
# You must create a config file, containing your databricks and github access tokens and
# the mlflow tracking experiment name. DON'T PUSH THIS FILE TO THE REMOTE REPO!
with open("../mlflow_config.yaml") as f:
    mlflow_config = yaml.safe_load(f)

# Set databricks env variables
os.environ["MLFLOW_TRACKING_URI"] = "databricks"
os.environ["DATABRICKS_HOST"] = mlflow_config["db_host"]
os.environ["DATABRICKS_TOKEN"] = mlflow_config["db_token"]

# Set github access uri
# user = mlflow_config["git_user"]
# token = mlflow_config["git_token"]
# git_uri = mlflow_config["git_uri"]
# uri = f"https://{user}:{token}@{git_uri}"
uri = (
    "/Users/alexmfalm/Documents/Git_Repos/basics-mlops-python"  # Local filesystem path
)
# git_branch = "main"

experiment_name = mlflow_config["exp_name"]
experiment = mlflow.set_experiment(experiment_name)
backend = "local"  # databricks

In [22]:
lrs = [0.001]  # , 0.0005, 0.0001, 0.00005, 0.00001, 0.000005, 0.000001]

for lr in lrs:
    hyperparams["lr"] = lr

    mlflow.projects.run(
        uri,
        # version=git_branch,
        entry_point="train",
        env_manager="local",
        parameters=hyperparams,
        backend=backend,
        backend_config=backend_config,
        experiment_id=experiment.experiment_id,
        synchronous=True,  # Set to False, if you don't want to wait for the model to train
    )

2022/09/28 14:32:49 INFO mlflow.projects.utils: === Created directory /var/folders/p6/5l5fzz397vq3lwg4c8p3t7lw0000gn/T/tmp995qjujp for downloading remote URIs passed to arguments of type 'path' ===
2022/09/28 14:32:49 INFO mlflow.utils.databricks_utils: No workspace ID specified; if your Databricks workspaces share the same host URL, you may want to specify the workspace ID (along with the host information in the secret manager) for run lineage tracking. For more details on how to specify this information in the secret manager, please refer to the Databricks MLflow documentation.
2022/09/28 14:32:49 INFO mlflow.projects.backend.local: === Running command 'python src/train.py \
  --resume_checkpoint None \
  --resume_run_id None \
  --max_epochs 10 \
  --gpus 0 \
  --batch_size 32 \
  --lr 0.001 \
  --num_samples -1 \
  --val_ratio 0.2 \
  --test_ratio 0 \
  --random_seed None \
  --dataset data/processed/dummy.json' in run with ID '3b6798fa4f504a71b5821453acec03fb' === 
2022/09/28 14:3

Training: 0it [00:00, ?it/s]                                                

/Users/alexmfalm/.local/share/virtualenvs/basics-mlops-python-VGYzWtQD/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   0%|          | 0/4 [00:00<?, ?it/s] 

[W reducer.cpp:1258] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())


...

Epoch 2, global step 9: 'epoch_val_score' was not in top 1


Epoch 3:  75%|███████▌  | 3/4 [00:20<00:06,  6.84s/it, loss=8.78e+16, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████| 4/4 [00:30<00:00,  7.72s/it, loss=8.78e+16, v_num=1]

Monitored metric epoch_val_score did not improve in the last 3 records. Best score: -29855.068. Signaling Trainer to stop.
Epoch 3, global step 12: 'epoch_val_score' was not in top 1


Epoch 3: 100%|██████████| 4/4 [00:31<00:00,  7.78s/it, loss=8.78e+16, v_num=1]


2022/09/28 14:35:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/alexmfalm/.local/share/virtualenvs/basics-mlops-python-VGYzWtQD/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2022/09/28 14:35:21 INFO mlflow.projects: === Run (ID '3b6798fa4f504a71b5821453acec03fb') succeeded ===
